# **Problem Statement and Modelling Objective**

Modelling Objective: Create ML models that can use meterological and/or soil data to predict drought incidents

Datasets: https://www.kaggle.com/datasets/cdminix/us-drought-meteorological-data?datasetId=1108326&sortBy=voteCount (rows and columns after dropping nulls and merging soil dataset: 678039 X 55)

# **Importing Required Modules**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import random
from pandas import option_context
import joblib

# **Loading the Datasets and Merging**
* Could not train models using the whole dataset due to memory errors, therefore to preserve memory-
    * Changed the float64 to float32 
    * Changed the date column to pd datetime
    * Used 70% of the dataset

In [ ]:
fp = r'drought_data\train_timeseries\train_timeseries.csv'

In [ ]:
dtypes = {
    'fips': 'int16',
    'PRECTOT': 'float32',
    'PS': 'float32',
    'QV2M': 'float32',
    'T2M': 'float32',
    'T2MDEW': 'float32',
    'T2MWET': 'float32',
    'T2M_MAX': 'float32',
    'T2M_MIN': 'float32',
    'T2M_RANGE': 'float32',
    'TS': 'float32',
    'WS10M': 'float32',
    'WS10M_MAX': 'float32',
    'WS10M_MIN': 'float32',
    'WS10M_RANGE': 'float32',
    'WS50M': 'float32',
    'WS50M_MAX': 'float32',
    'WS50M_MIN': 'float32',
    'WS50M_RANGE': 'float32',
    'score': 'float32'
}

sample_fraction = 0.7
random.seed(45)

def skip_row(row_idx):
  if row_idx == 0:
    return False
  return random.random() > sample_fraction

In [ ]:
drought_df = pd.read_csv(
    r'drought_data\train_timeseries\train_timeseries.csv',
    parse_dates= ['date'],
    dtype=dtypes,
    skiprows=skip_row 
    )

In [ ]:
soil_df = pd.read_csv(r'drought_data\soil_data.csv')

In [ ]:
drought_df.shape

# **Exploratory Data Analysis**

The drought scores are available weekly while the daily meteorological data points are given. Therfore removing the rows where score (target column) is null.

In [ ]:
drought_df = drought_df.dropna()
drought_df['score'] = drought_df['score'].round().astype(int)

Merging the soil data with the drought dataset using 'fips' column present in both datasets which contains unique id for each US county.

In [ ]:
drought_df_soil = drought_df.merge(
    soil_df, 
    left_on='fips', 
    right_on='fips'
    )

In [ ]:
drought_df_soil.describe()

In [ ]:
drought_df_soil.shape

Looking at the distribution of Score

In [ ]:
px.histogram(drought_df_soil, x='score')

Looking at correlation between different features

In [ ]:
all_corr = drought_df_soil.select_dtypes(np.number).corr()
corr_sub = all_corr.sort_values(by='score', ascending=False).head(25)
sns.heatmap(corr_sub[list(corr_sub.index)], annot=False, cmap='Reds')

In [ ]:
corr_sub = all_corr.sort_values(by='score', ascending=False).head(15)
sns.heatmap(corr_sub[list(corr_sub.index)], annot=False, cmap='Reds')

Checking for outliers

In [ ]:
plt.figure(figsize=(20,10))
drought_df_soil.iloc[:, 1:10].boxplot()       


In [ ]:
plt.figure(figsize=(20,10))
drought_df_soil.iloc[:, 10:20].boxplot()    

In [ ]:
plt.figure(figsize=(20,10))
drought_df_soil.iloc[:, 20:30].boxplot()   

In [ ]:
plt.figure(figsize=(20,10))
drought_df_soil.iloc[:, 30:40].boxplot()   

In [ ]:
plt.figure(figsize=(20,10))
drought_df_soil.iloc[:, 40:-3].boxplot()   

In [ ]:
list(corr_sub.index)

In [ ]:
px.scatter(drought_df_soil.sample(n=1000), x='TS', y='T2M_MAX', color='score', opacity=0.8)

In [ ]:
px.scatter(drought_df_soil.loc[drought_df_soil.score != 0].sample(n=1000), x='T2M', y='T2M_MAX', color='score', opacity=0.8)

In [ ]:
px.histogram(drought_df_soil.loc[drought_df_soil.score != 0], x= 'GRS_LAND', color='score')

In [ ]:
drought_df_soil.GRS_LAND.describe()

In [ ]:
px.histogram(drought_df_soil.loc[(drought_df_soil.score != 0) & (drought_df_soil.elevation < 500)], x= 'elevation', color='score')

In [ ]:
px.histogram(drought_df_soil.loc[(drought_df_soil.PRECTOT < 5) & (drought_df_soil.PRECTOT > 0.2)], x= 'PRECTOT', color='score')

In [ ]:
px.histogram(drought_df_soil.loc[drought_df_soil.score != 0], x= 'slope1', color='score')

In [ ]:
drought_df_soil.columns

In [ ]:
px.scatter(drought_df_soil.sample(n=1000), x= 'WS10M', y='WS50M', color='score', opacity=0.8)

# **Feature Engineering**

In EDA, we saw that the columns have a lot of outliers that may affect the accuracy. Using standard deviation and mean to identify outliers.

In [ ]:
drought_df_soil.score.isnull().sum()

In [ ]:
drought_df_soil.score.value_counts()

In [ ]:
cols_ex_fipsNscore = list(drought_df.loc[:, ~drought_df.columns.isin(['score', 'fips', 'date'])].columns)

In [ ]:
cols_ex_fipsNscore_soil = list(drought_df_soil.loc[:, ~drought_df_soil.columns.isin(['score', 'fips', 'date'])].columns)

In [ ]:
def remove_outliers(df, columns, n_std=3):
    for col in columns:
        # print('---------------------------------------')
        # print(f'Removing outliers from column: {col}')

        mean = df[col].mean()
        sd = df[col].std()

        df = df[(df[col] <= mean + (n_std * sd))]
        df = df[(df[col] >= mean - (n_std * sd))]

    return df

In [ ]:
drought_df = remove_outliers(drought_df, cols_ex_fipsNscore)

In [ ]:
drought_df_soil = remove_outliers(drought_df_soil, columns= cols_ex_fipsNscore_soil)

In [ ]:
drought_df_soil.isnull().sum()

Drought is time/season dependent, I am also going to use day, month and year to train models.

In [ ]:
drought_df['year'] = drought_df.date.dt.year
drought_df['month'] = drought_df.date.dt.month
drought_df['day'] = drought_df.date.dt.day

In [ ]:
drought_df_soil['year'] = drought_df_soil.date.dt.year
drought_df_soil['month'] = drought_df_soil.date.dt.month
drought_df_soil['day'] = drought_df_soil.date.dt.day

I am going to train models only using meterological data first and then with both metereological and soil data to compare the accuracies

In [ ]:
scaler = StandardScaler()

In [ ]:
x_inputs = drought_df[cols_ex_fipsNscore + ['year', 'month', 'day']]
x_inputs[cols_ex_fipsNscore + ['year', 'month', 'day']] = scaler.fit_transform(x_inputs[cols_ex_fipsNscore + ['year', 'month', 'day']])
target = drought_df['score']

In [ ]:
x_inputs_soil = drought_df_soil[cols_ex_fipsNscore_soil+['year', 'month', 'day']]
x_inputs_soil[cols_ex_fipsNscore_soil+['year', 'month', 'day']] = scaler.fit_transform(x_inputs_soil[cols_ex_fipsNscore_soil+['year', 'month', 'day']])
target_soil = drought_df_soil['score']

In [ ]:
x_inputs_soil.head()

Using sklearn.feature_selection.RFE to identify the identify the 24 most important features/columns for predicting drought scores

In [ ]:
rfe_model = RandomForestClassifier(n_estimators=10)
rfe = RFE(rfe_model, n_features_to_select=24)
rfe.fit(x_inputs_soil, target_soil)

In [ ]:
feature_df = pd.DataFrame(list(x_inputs_soil.columns), columns=['features'])
feat_selection_df = pd.DataFrame(rfe.support_, columns=['selection'])
feat_selection_df_merge = pd.merge(feature_df, feat_selection_df, left_index=True, right_index=True)
print(feat_selection_df_merge.sort_values('selection', ascending=False).head(24))
print(list(feat_selection_df_merge.sort_values('selection', ascending=False).head(24).features))

In [ ]:
col_select = [
    'PRECTOT',
    'WS50M_MIN',
    'month',
    'WS10M_MAX',
    'T2MWET',
    'PS',
    'QV2M',	
    'T2M',	
    'T2MDEW',	
    'T2M_MAX',	
    'T2M_MIN',	
    'T2M_RANGE',	
    'TS',	
    'WS10M',	
    'WS10M_RANGE',	
    'WS50M',	
    'WS50M_MAX',	
    'WS50M_RANGE',
    'year',
    'day'
    ]
# col_select_soil = ['PRECTOT',
#  'WS10M_RANGE',
#  'PS',
#  'slope1',
#  'elevation',
#  'lon',
#  'lat',
#  'day',
#  'year',
#  'WS50M_RANGE',
#  'WS50M_MIN',
#  'WS50M_MAX',
#  'WS50M',
#  'month',
#  'T2M_MAX',
#  'QV2M',
#  'T2M',
#  'T2MDEW',
#  'T2MWET',
#  'GRS_LAND',
#  'T2M_MIN',
#  'T2M_RANGE',
#  'TS',
#  'WS10M']

col_select_soil= [
    'PRECTOT', 'WS10M_RANGE', 'month', 'year', 
    'GRS_LAND', 'PS', 'elevation', 'lon', 'lat', 
    'WS50M_RANGE', 'WS50M_MIN', 'WS50M_MAX', 'WS50M', 
    'day', 'WS10M', 'TS', 'T2M_RANGE', 'T2M_MIN', 'T2M_MAX', 
    'QV2M', 'T2M', 'WS10M_MAX', 'T2MDEW', 'T2MWET']

Splitting data for traing and testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
                                    x_inputs[col_select], 
                                    target, 
                                    test_size=0.2, 
                                    random_state=45
                                    )

In [ ]:
x_train_soil, x_test_soil, y_train_soil, y_test_soil = train_test_split(
                                    x_inputs_soil[col_select_soil], 
                                    target_soil, 
                                    test_size=0.2, 
                                    random_state=45
                                    )

In EDA, we saw that there is class imbalance in the dataset, i.e. we have a lot of rows where the drought score (target) is 0. Therefore, I am using imblearn.over_sampling.SMOTE to upsample the rows where the drought score is 1, 2, 3, 4 or 5.

In [ ]:
sm = SMOTE(random_state = 45)

In [ ]:
x_train_smote, y_train_smote = sm.fit_resample(x_train, y_train.ravel())
x_test_smote, y_test_smote = sm.fit_resample(x_test, y_test.ravel())

In [ ]:
x_train_smote.shape, y_train_smote.shape, x_test_smote.shape, y_test_smote.shape

In [ ]:
x_train_soil_smote, y_train_soil_smote = sm.fit_resample(x_train_soil, y_train_soil.ravel())
x_test_soil_smote, y_test_soil_smote = sm.fit_resample(x_test_soil, y_test_soil.ravel())

In [ ]:
x_train_soil_smote.shape, y_train_soil_smote.shape, x_test_soil_smote.shape, y_test_soil_smote.shape

In [ ]:
dup = {x for x in list(x_train_soil_smote.columns) if list(x_train_soil_smote.columns).count(x) > 1}
print(dup)

In [ ]:
x_train_soil_smote.head()

# **Modelling, Predicting and Hyperparameter Tuning**

* Decision Tree
* Random Forest Classifier
* Gradient Boosting

I will train models with original data i.e., without upsampling, after upsampling, with meterological data and finally with metreological + soil data and observe the results. 

# *Model #1: Decision Trees*

In [ ]:
def dt_test_params(x_tr, y_tr, x_tes, y_tes, **params):
    model = DecisionTreeClassifier(random_state=45, **params).fit(x_tr, y_tr)
    test_preds = model.predict(x_tes)
    train_preds = model.predict(x_tr)

    train_acc = accuracy_score(y_tr, train_preds)
    test_acc = accuracy_score(y_tes, test_preds)
    test_kappa = cohen_kappa_score(y_tes, test_preds)

    cf = confusion_matrix(y_tes, test_preds, normalize='true')
    plt.figure()
    sns.heatmap(cf, annot=True, cmap= 'Greens')
    plt.xlabel('Predictions')
    plt.xlabel('Target')
    plt.title('Confusion Matrix')

    
    return train_acc, test_acc, test_kappa, model

1. Using unbalanced dataset

In [ ]:
acc = []
for i in [20, 50, 100, 150]:
    print(f"max_depth: {i}")
    acc.append(dt_test_params(
        x_train, 
        y_train,
        x_test,
        y_test, 
        max_depth = i
        ))

In [ ]:
results1 = pd.DataFrame(acc, columns=['train_acc', 'test_acc', 'kappa', 'model'])
plt.figure(figsize=(10,6))
plt.title('Overfitting curve for selected parameters')
plt.plot([20, 50, 100, 150], 1- results1.train_acc, 'b-o')
plt.plot([20, 50, 100, 150], 1- results1.test_acc, 'g-o')
#plt.plot(50, acc[1], 'r-o')
plt.xlabel('Hyperparameter Values')
plt.ylabel('Error')
plt.legend(['Training Error', 'Test Error'])

2. Using balanced dataset computed using SMOTE

In [ ]:
acc1 = []
for i in [20, 50, 100, 150]:
    print(f"max_depth: {i}")
    acc1.append(dt_test_params(
        x_train_smote, 
        y_train_smote,
        x_test,
        y_test, 
        max_depth = i
        ))

In [ ]:
results2 = pd.DataFrame(acc1, columns=['train_acc', 'test_acc', 'kappa', 'model'])
plt.figure(figsize=(10,6))
plt.title('Overfitting curve for selected parameters')
plt.plot([20, 50, 100, 150], 1- results2.train_acc, 'b-o')
plt.plot([20, 50, 100, 150], 1- results2.test_acc, 'g-o')
#plt.plot(50, acc[1], 'r-o')
plt.xlabel('Hyperparameter Values')
plt.ylabel('Error')
plt.legend(['Training Error', 'Test Error'])

3. Using balanced dataset which includes both metereological and soil dataset

In [ ]:
acc2 = []
for i in [25, 50, 75, 100]:
    print(f"max_depth: {i}")
    acc2.append(dt_test_params(
        x_tr = x_train_soil_smote, 
        y_tr = y_train_soil_smote,
        x_tes = x_test_soil,
        y_tes = y_test_soil, 
        max_depth = i
        ))

In [ ]:
results3 = pd.DataFrame(acc2, columns=['train_acc', 'test_acc', 'kappa', 'model'])
plt.figure(figsize=(10,6))
plt.title('Overfitting curve for selected parameters')
plt.plot([20, 50, 100, 150], 1- results3.train_acc, 'b-o')
plt.plot([20, 50, 100, 150], 1- results3.test_acc, 'g-o')
#plt.plot(50, acc[1], 'r-o')
plt.xlabel('Hyperparameter Values')
plt.ylabel('Error')
plt.legend(['Training Error', 'Test Error'])

Hypertuning 3. Using balanced dataset which includes both metereological and soil dataset

Using sklearn.model_selection.GridSearchCV to compare different hyperparameters

In [ ]:
params = {
    'max_depth': [30, 50, 100],
    'min_samples_leaf': [10, 30, 65, 100],
    'max_features': ['sqrt', 'log2', None]
}

In [ ]:
grid_search_dt = GridSearchCV(
                        estimator=DecisionTreeClassifier(random_state=45),
                        param_grid=params, 
                        cv=4, 
                        n_jobs=-1, 
                        verbose=1, 
                        scoring = "accuracy"
                        )

In [ ]:
grid_search_dt.fit(pd.DataFrame(x_train_soil_smote).sample(frac=0.5, random_state=45), pd.DataFrame(y_train_soil_smote).sample(frac=0.5, random_state=45)) 

In [ ]:
score_df_dt = pd.DataFrame(grid_search_dt.cv_results_)
with option_context('display.max_colwidth', None):
    # display the dataframe
    display(score_df_dt.sort_values('mean_test_score', ascending=False).head(5))

In [ ]:
dt_test_params(
        x_tr = x_train_soil_smote, 
        y_tr = y_train_soil_smote,
        x_tes = x_test_soil,
        y_tes = y_test_soil, 
        max_depth =  100, 
        max_features= None, 
        min_samples_leaf= 10
        )

# *Model #2: Random Forest*

In [ ]:
def rf_test_params(x_tr, y_tr, x_tes, y_tes, **params):
    model = RandomForestClassifier(random_state=45, **params).fit(x_tr, y_tr)
    test_preds = model.predict(x_tes)

    #train_acc = accuracy_score(y_train_smote, train_preds)
    test_acc = accuracy_score(y_tes, test_preds)
    test_kappa = cohen_kappa_score(y_tes, test_preds)

    cf = confusion_matrix(y_tes, test_preds, normalize='true')
    plt.figure()
    sns.heatmap(cf, annot=True, cmap= 'Greens')
    plt.xlabel('Predictions')
    plt.xlabel('Target')
    plt.title('Confusion Matrix')

    
    return [test_acc, test_kappa, model]

1. Random Forest model with metereological and soil data upsampled (Not going to try data with imbalanced classes since the results from balanced datasets were better with decision trees classifier)

In [ ]:
rf_test_params(
    x_train_soil_smote, 
    y_train_soil_smote,
    x_test_soil,
    y_test_soil, 
    max_depth = 50,
    n_estimators = 100
)

Hyperparameter Tuning 1. Random Forest model with metereological and soil data upsampled

Using sklearn.model_selection.RandomizedSearchCV to find appropriate hyperparameters (Tried GridSearchCV first but was it was taking to long)

In [ ]:
params_rf = {
    'n_estimators': [20, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [10, 50, 100, 150],
    'bootstrap': [True, False]
    }

In [ ]:
rand_search_rf = RandomizedSearchCV(
                        estimator=RandomForestClassifier(random_state=45, n_estimators=50),
                        param_distributions=params_rf,
                        n_iter=25, 
                        cv=3, 
                        n_jobs=-1, 
                        verbose=20, 
                        random_state=45
                        )

In [ ]:
pd.DataFrame(y_train_soil_smote).sample(frac=0.05, random_state=45).to_numpy().ravel()

In [ ]:
rand_search_rf.fit(
    pd.DataFrame(x_train_soil_smote).sample(frac=0.05, random_state=45), 
    pd.DataFrame(y_train_soil_smote).sample(frac=0.05, random_state=45).to_numpy().ravel()
    )

In [ ]:
rand_search_rf.best_params_
# score_df_rf = pd.DataFrame(rand_search_rf.cv_results_)
# with option_context('display.max_colwidth', None):
#     display(score_df_rf.sort_values('mean_test_score', ascending=False).head(5))

In [ ]:
rf_test_params(
    x_train_soil_smote, 
    y_train_soil_smote,
    x_test_soil,
    y_test_soil, 
    n_estimators = 50,
    max_features= None,
    max_depth = 100,
    bootstrap= True
)

# *Model #3: XGBoost*

In [ ]:
def xgb_test_params(x_tr, y_tr, x_tes, y_tes, **params):
    model = XGBClassifier(random_state=45, n_jobs=-1, **params).fit(x_tr, y_tr)
    test_preds = model.predict(x_tes)

    #train_acc = accuracy_score(y_train_smote, train_preds)
    test_acc = accuracy_score(y_tes, test_preds)
    test_kappa = cohen_kappa_score(y_tes, test_preds)

    cf = confusion_matrix(y_tes, test_preds, normalize='true')
    plt.figure()
    sns.heatmap(cf, annot=True, cmap= 'Greens')
    plt.ylabel('Predictions')
    plt.xlabel('Target')
    plt.title('Confusion Matrix')

    
    return test_acc, test_kappa, model

In [ ]:
xgb_test_params(
    pd.DataFrame(x_train_soil_smote).sample(frac=0.4, random_state=45), 
    pd.DataFrame(y_train_soil_smote).sample(frac=0.4, random_state=45),
    x_test_soil,
    y_test_soil, 
    n_estimators = 100,
    max_depth = 50,
    learning_rate = 0.9,
    verbosity = 1,
    objective='multi:softprob',
    subsample = 0.8,
    colsample_bytree = 0.8
)

Hyperparameter tuning

In [ ]:
params_xgb = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 25, 50, 100],
    'learning_rate': [0.1, 0.3, 0.7, 0.9],
    'subsample': [0.5, 0.8],
    'colsample_bytree': [0.5, 0.8]
    }

In [ ]:
rand_search_xgb = RandomizedSearchCV(
                        estimator=XGBClassifier(random_state=45, objective = 'multi:softprob'),
                        param_distributions=params_xgb,
                        n_iter=25, 
                        cv=3, 
                        n_jobs=-1, 
                        verbose=2, 
                        random_state=45
                        )

In [ ]:
rand_search_xgb.fit(
    pd.DataFrame(x_train_soil_smote).sample(frac=0.05, random_state=45), 
    pd.DataFrame(y_train_soil_smote).sample(frac=0.05, random_state=45)
    )

In [ ]:
rand_search_xgb.best_params_

In [ ]:
xgb_test_params(
    pd.DataFrame(x_train_soil_smote).sample(frac=0.4, random_state=45), 
    pd.DataFrame(y_train_soil_smote).sample(frac=0.4, random_state=45),
    x_test_soil,
    y_test_soil, 
    n_estimators = 150,
    max_depth = 25,
    learning_rate = 0.1,
    verbosity = 1,
    objective='multi:softprob',
    subsample = 0.8,
    colsample_bytree = 0.8
)

# **Performance of the Best Models**

In [3]:
all_res = {
    'Classifiers': ['Decision Trees', 'Random Forest', 'Gradient Boosting'],
    'Hyperparameters Used': 
        [
            'max_depth: 100, max_features: None, min_samples_leaf: 10',
            'n_estimators: 50, max_features: None, max_depth: 100, bootstrap: True',
            'n_estimators: 150, max_depth: 25, learning_rate: 0.1, objective: multi:softprob, subsample: 0.8, colsample_bytree: 0.8'
        ],
    'Test Accuracy': [0.8310, 0.8902, 0.8768],
    'Kappa Score': [0.7108, 0.8082, 0.7842],
    'Training and Prediction Time (min)': [2, 87, 21]
}
with option_context('display.max_colwidth', None):
    display(pd.DataFrame(all_res).sort_values('Test Accuracy', ascending=False))

,Classifiers,Hyperparameters Used,Test Accuracy,Kappa Score,Training and Prediction Time (min)
1,Random Forest,"n_estimators: 50, max_features: None, max_depth: 100, bootstrap: True",0.8902,0.8082,87
2,Gradient Boosting,"n_estimators: 150, max_depth: 25, learning_rate: 0.1, objective: multi:softprob, subsample: 0.8, colsample_bytree: 0.8",0.8768,0.7842,21
0,Decision Trees,"max_depth: 100, max_features: None, min_samples_leaf: 10",0.8310,0.7108,2


<p>Confusion Matrix- Random Forest</p>
<img src="confusion/rf.png"/>

<p>Confusion Matrix- Gradient Boosting</p>
<img src="confusion/xgb.png"/>

<p>Confusion Matrix- Decicion Trees</p>
<img src="confusion/dt.png" />

# *Sample Predictions of Final Model*

In [ ]:
x_samp_test = x_test_soil.sample(random_state=105, n=100).reset_index(drop=True)
y_samp_test = pd.DataFrame(y_test_soil.sample(random_state=105, n=100)).reset_index(drop=True)

In [ ]:
final_model =   RandomForestClassifier(
                n_jobs=-1,
                random_state=45,
                n_estimators = 50,
                max_features= None,
                max_depth = 100,
                bootstrap= True
            )
final_model.fit(x_train_soil_smote, y_train_soil_smote)

In [ ]:
sample_predictions = final_model.predict(x_samp_test)

In [ ]:
sample_predictions = pd.DataFrame(sample_predictions, columns=['Model Predictions']).merge(
    y_samp_test.rename(columns = {'score':'Target Scores'}), 
    left_index=True, 
    right_index=True
    )

In [ ]:
sample_predictions.sort_values('Target Scores', ascending=False).head(20)

In [ ]:
print(f"Total Samples: {len(y_samp_test)}")
print(f"Total Correct Predictions: {accuracy_score(y_samp_test, sample_predictions['Model Predictions'], normalize = False)}")
print(f"Total Incorrect Predictions: {len(y_samp_test) - accuracy_score(y_samp_test, sample_predictions['Model Predictions'], normalize = False)}")
print(f"Sample Prediction Accuracy: {accuracy_score(y_samp_test, sample_predictions['Model Predictions'])}")

plt.figure()
sns.heatmap(confusion_matrix(y_samp_test, sample_predictions['Model Predictions'], normalize= None), annot=True, cmap= 'Greens')
plt.ylabel('Predictions')
plt.xlabel('Target')
plt.title('Confusion Matrix')

# **Saving the Best Model**

In [ ]:
def date_converter(df, date_col_name):
    if df.date_col_name.dtype != 'datetime64[ns]':
        df['pd_datetime'] = pd.to_datetime(df.date_col_name)
        df['year'] = df.pd_datetime.dt.year
        df['month'] = df.pd_datetime.dt.month
        df['day'] = df.pd_datetime.dt.day
    else:
        df['year'] = df.date_col_name.dt.year
        df['month'] = df.date_col_name.dt.month
        df['day'] = df.date_col_name.dt.day

In [ ]:
def my_imputer(df):
    ask = input('Want to drop all rows with nulls? (y/n)')
    if ask == 'y':
        df = df.dropna()
    elif ask == 'n':
        pass
    else:
        print('Error: Invalid Input, operation not completed')

In [ ]:
def soil_data_merge(df, soil_df, merge_col_left, merge_col_right):
    df = drought_df.merge(
    soil_df, 
    left_on=merge_col_left, 
    right_on=merge_col_right
    )

In [ ]:
drought_prediction_RF_Model = {
    'model': final_model,
    'scaler': scaler,
    'input_cols': col_select_soil,
    'target_col': 'score',
    'imputer': my_imputer,
    'date_converter': date_converter,
    'merge_soil_data': soil_data_merge
}

In [ ]:
joblib.dump(drought_prediction_RF_Model, 'drought_prediction_RF_Model.joblib')

# **Summary and Recommendations**

Summary:
<br>
Random Forest classifier (RF) trained with upsampled metereological and soil data was able to produce higher accuracy than Decision Trees classifier (DT) and gradient boosting (XGB) (Note: XGB classifier was only trained with 40% of the data). Results of hypertuned RF classifier was significantly better than hypertuned DT classifier but it took almost 40 times more time than DT classifier to train and make predictions. Eventhough XGB classifier was trained with only 40% of the data,results produced by it and RF classifier were comparable.  
<br>
<br>
Observations Regarding Data:
* Observation 1: Dataset was imbalanced i.e. drought score of 0 was significantly higher in numbers than other scores.
<br>
<br>
Treatment: Oversampling/upsampling was done to balance the dataset.
<br>
Treatment Result: Accuracy improved

* Observation 2: Dataset had many outliers
<br>
<br>
Treatment: Outliers were removed using standard deviation and mean
<br>
Treatment Result: Unknown (Did not train model using outliers)

* Observation 3: Columns related to temperature and wind speed had high correlation.
<br>
<br>
Treatment: None

<br>
<br>
Suggestions/Recommendations/Questions for Future Work:
 
* Was removing outliers a good idea? Did it affect the accuracy?
* How would combining temerature and windspeed related columns (for eg. using PCA) affect the accuracy? Will that reduce the training time significantly?
* If XGB classifier was trained with the whole dataset, would it produce better results than RF classifier.


# **References**

* Reference Notebook: https://www.kaggle.com/code/akshayasrinivasan2/drought-prediction-using-ml-algorithms
* Feature Selection: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html
* Feature Selection: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html
* Dealing with Unbalanced Datasets/Oversampling: https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html
* XGBoost Reference: https://xgboost.readthedocs.io/en/stable/parameter.html
* XGBoost Hyperparameter Tuning: https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
* Dealing with Outliers: https://medium.com/analytics-vidhya/how-to-remove-outliers-for-machine-learning-24620c4657e8